In [1]:
import pandas as pd
import numpy as np
import requests
from requests.auth import HTTPBasicAuth 
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from itertools import groupby
from operator import itemgetter
from sklearn.linear_model import LinearRegression

In [ ]:
# Load data into a data frame
url = 'https://www.quandl.com/api/v3/datasets/FSE/BDT_X?api_key=uBMJxSz4ayVn-RhgK3aT'
r = requests.get(url)
json = r.json()
column_names = json['dataset']['column_names']
df = pd.DataFrame(json['dataset']['data'])
df = df.rename(columns={0:column_names[0], 1:column_names[1], 2:column_names[2], 3:column_names[3], 4:column_names[4], 5:column_names[5], 6:column_names[6], 7:column_names[7], 8:column_names[8], 9:column_names[9], 10:column_names[10]})
df

In [ ]:
# Display the 90,30,7 days moving average on a plot graph.
# Assumption 1: On closing price
# Assumption 2: Time period is 12 months
start_date = "2020-01-01"
end_date = "2020-11-30"
after_start_date = df["Date"] >= start_date
before_end_date = df["Date"] <= end_date
between_two_dates = after_start_date & before_end_date
df = df.filter(items=['Date', 'Close'])
df = df.loc[between_two_dates]
df = df.iloc[::-1]

plt.plot(df.Date, df.Close)
tick_spacing = 30

# Set the locator
locator = mdates.MonthLocator()  # every month
# Specify the format - %b gives us Jan, Feb...
fmt = mdates.DateFormatter('%b')
X = plt.gca().xaxis
X.set_major_locator(locator)
# Specify formatter
X.set_major_formatter(fmt)

plt.show()

In [ ]:
moving_average_7 = df.Close.rolling(window=7).mean()
moving_average_30 = df.Close.rolling(window=30).mean()
moving_average_90 = df.Close.rolling(window=90).mean()
monthly_average = df.groupby(pd.PeriodIndex(df['Date'], freq="M"))['Close'].mean() 
monthly_average.index = monthly_average.index.to_timestamp()

df['monthly_average'] = df['Close']
df['monthly_average'] = np.nan
for index in monthly_average.index:
    date = index
    while( (df.loc[df['Date'] == date.strftime('%Y-%m-%d')]).empty ):
        date += pd.Timedelta("1 day")
    df.loc[df['Date'] == date.strftime('%Y-%m-%d'), "monthly_average"] = np.round(monthly_average.get(index),2)
    

#Detect and present (mark on the graph with a symbol, e.g. green triangle) 
#the dates where there are 5 consecutive days of prices going up.
df['price_up_markers'] = df['Close']
price_up = (df.Close.diff() > 0)

price_up = np.concatenate(([False],price_up,[False]))
idx = np.flatnonzero(price_up)

price_up_list = []
for k, g in groupby(enumerate(idx), lambda ix : ix[0] - ix[1]):
    up_list = list(map(itemgetter(1), g))
    if len(up_list) >= 5:
        price_up_list.extend(up_list)

for index in range(1, len(df['price_up_markers'])):
    if index not in price_up_list:
        df.loc[index, 'price_up_markers'] = np.nan
        

#Detect and present the dates where there are 4 consecutive days of prices going down (different symbol).
df['price_down_markers'] = df['Close']
price_down = (df.Close.diff() < 0)

price_down = np.concatenate(([False],price_down,[False]))
idx = np.flatnonzero(price_down)

price_down_list = []
for k, g in groupby(enumerate(idx), lambda ix : ix[0] - ix[1]):
    down_list = list(map(itemgetter(1), g))
    if len(down_list) >= 4:
        price_down_list.extend(down_list)

for index in range(1, len(df['price_down_markers'])):
    if index not in price_down_list:
        df.loc[index, 'price_down_markers'] = np.nan
        
# Plots
plt.plot(df.Date, df.Close, label='Closing Price')
plt.plot(df.Date, moving_average_7, label='7 day moving average', color='orange')
plt.plot(df.Date, moving_average_30, label='30 day moving average', color='brown')
plt.plot(df.Date, moving_average_90, label='90 day moving average', color='purple')
plt.plot(df.Date, df.monthly_average, 'bo')
plt.plot(df.Date, df.price_up_markers, 'g^' )
plt.plot(df.Date, df.price_down_markers, 'ro')
plt.legend(loc='upper right')


# Set the locator for every month
locator = mdates.MonthLocator()  
# Specify the format - %b gives us Jan, Feb...
fmt = mdates.DateFormatter('%b')
X = plt.gca().xaxis
X.set_major_locator(locator)
# Specify formatter
X.set_major_formatter(fmt)


plt.show()



In [ ]:
# Calculate and show on the graph a regression line for the last 90, 30 and 7 days in the plot.
# X is dates
# Y is closing prices
# Set the locator for every month

# 7 day
df_7_day = df.tail(7)

y = df_7_day['Close']     # create a new y-col (optional)
x = np.arange(len(df_7_day))  # create x-col of continuous integers

m, b = np.polyfit(x, y, 1)
plt.plot(x, m*x + b, label='7 day regression line', color='blue')
plt.scatter(x,y)

plt.legend(loc='upper left')
plt.show()

In [ ]:
# 7 day
df_30_day = df.tail(30)

y = df_30_day['Close']     # create a new y-col (optional)
x = np.arange(len(df_30_day))  # create x-col of continuous integers

m, b = np.polyfit(x, y, 1)
plt.plot(x, m*x + b, label='30 day regression line', color='blue')
plt.scatter(x,y)
plt.legend(loc='upper left')
plt.show()

In [ ]:
# 90 day
df_90_day = df.tail(90)

y = df_90_day['Close']     # create a new y-col (optional)
x = np.arange(len(df_90_day))  # create x-col of continuous integers

m, b = np.polyfit(x, y, 1)
plt.plot(x, m*x + b, label='90 day regression line', color='blue')
plt.scatter(x,y)
plt.legend(loc='upper left')
plt.show()